In [4]:
from foreduce.tptp.parser import *

p = read_string("""
%------------------------------------------------------------------------------
% File     : PUZ128+2 : TPTP v8.2.0. Released v4.0.0.
% Domain   : Puzzles
% Problem  : Iokaste patricide triangle
% Version  : Especial.
%            Theorem formulation : Converted from ACE by the APE [FKK08].
% English  : Iokaste is a parent of Oedipus. Iokaste is a parent of Polyneikes.
%            Oedipus is a parent of Polyneikes. Polyneikes is a parent of 
%            Thersandros. Oedipus is a patricide. Thersandros is not a 
%            patricide. Therefore, Iokaste is a parent of a patricide that is 
%            a parent of somebody who is not a patricide.

% Refs     : [FKK08] Fuchs et al. (2008), Attempto Controlled English for K
% Source   : [TPTP]
% Names    : 

% Status   : Theorem
% Rating   : 0.22 v8.2.0, 0.25 v8.1.0, 0.14 v7.5.0, 0.16 v7.4.0, 0.17 v7.3.0, 0.21 v7.1.0, 0.17 v7.0.0, 0.20 v6.4.0, 0.27 v6.3.0, 0.12 v6.2.0, 0.20 v6.0.0, 0.17 v5.5.0, 0.33 v5.4.0, 0.32 v5.3.0, 0.41 v5.2.0, 0.20 v5.1.0, 0.19 v5.0.0, 0.29 v4.1.0, 0.39 v4.0.1, 0.43 v4.0.0
% Syntax   : Number of formulae    :    2 (   0 unt;   0 def)
%            Number of atoms       :   26 (   9 equ)
%            Maximal formula atoms :   16 (  13 avg)
%            Number of connectives :   26 (   2   ~;   0   |;  24   &)
%                                         (   0 <=>;   0  =>;   0  <=;   0 <~>)
%            Maximal formula depth :   23 (  18 avg)
%            Maximal term depth    :    1 (   1 avg)
%            Number of predicates  :    5 (   3 usr;   1 prp; 0-3 aty)
%            Number of functors    :    5 (   5 usr;   5 con; 0-0 aty)
%            Number of variables   :   11 (   0   !;  11   ?)
% SPC      : FOF_THM_RFO_SEQ

% Comments : 
%------------------------------------------------------------------------------
fof(background,axiom,
    ? [A,B,C,D,E] :
      ( parent(A)
      & relation(A,of,'Oedipus')
      & 'Iokaste' = A
      & parent(B)
      & relation(B,of,'Polyneikes')
      & 'Iokaste' = B
      & parent(C)
      & relation(C,of,'Polyneikes')
      & 'Oedipus' = C
      & parent(D)
      & relation(D,of,'Thersandros')
      & 'Polyneikes' = D
      & patricide(E)
      & 'Oedipus' = E
      & ~ ? [F] :
            ( patricide(F)
            & 'Thersandros' = F ) ) ).

fof(prove,conjecture,
    ? [A,B,C,D] :
      ( parent(A)
      & patricide(B)
      & parent(C)
      & $true
      & ~ ? [E] :
            ( patricide(E)
            & D = E )
      & relation(C,of,D)
      & B = C
      & relation(A,of,B)
      & 'Iokaste' = A ) ).

%------------------------------------------------------------------------------

""")
p

In [5]:
from foreduce.prover.state import State

state = State(p)

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987296916/work/aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [1]:
VAMPIRE_PATH = '/home/apluska/.vampire/bin/vampire_z3_rel_static_casc2023_6749'
TPTP_PATH = '/home/apluska/TPTP-v8.2.0/'

In [2]:
from foreduce.tptp.parser import read_file
import os
from tqdm.auto import tqdm
from itertools import zip_longest

total, success = 0, 0
variables, functions = 0, []
problems = []
for dir in (pbar := tqdm(os.listdir(TPTP_PATH + 'Problems'))):
    for file in os.listdir(TPTP_PATH + 'Problems/' + dir):
        current = file
        pbar.set_description(f'Selected {success}/{total} Problems, currently parsing {dir}/{file}')
        if not file.endswith('.p'):
            continue
        try:
            total += 1
            problem = read_file(TPTP_PATH + 'Problems/' + dir + '/' + file, include_path=TPTP_PATH, max_size=100_000)
            success += 1
            variables = max(variables, len(problem.variables()))
            _functions = []
            for f in problem.function_symbols() + problem.predicate_symbols():
                if len(_functions) <= f.arity:
                    _functions += [0 for _ in range(f.arity - len(_functions))]
                _functions[f.arity] += 1
            functions = [max(a, b) for a, b in zip_longest(functions, _functions, fillvalue=0)]
            problems.append(dir + '/' + file)
        except Exception as e:
            continue
pbar.set_description(f'Selected {success}/{total} Problems')

with open('problems.txt', 'w') as f:
    f.write('\n'.join(problems))

print(f'Maximum number of variables: {variables}')
print(f'Maximum number of functions of each arity: {functions}')

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Succesfully parsed 8954/25473 Problems, currently parsing SEV/SEV514^1.p: 100%|██████████| 55/55 [03:17<00:00,  3.60s/it]         


In [14]:
import subprocess

with open('problems.txt', 'r') as f:
    problems = f.read().split('\n')

total, success = 0, 0
for problem in (pbar := tqdm(problems)):
    pbar.set_description(f'Succesfully proved {success}/{total} Problems, currently proving {problem}')
    args = [VAMPIRE_PATH, TPTP_PATH + 'Problems/' + problem,  '--show_new', 'on', '--include', TPTP_PATH, '-t', '1', '--avatar', 'off', '--proof', 'off']
    try:
        result = subprocess.run(args, capture_output=True, text=True, timeout=5)
    except subprocess.TimeoutExpired:
        continue
    if result.returncode == 0:
        success += 1    
        os.makedirs(os.path.dirname('./proofs/' + problem), exist_ok=True)
        with open('./proofs/' + problem, 'w') as f:
            f.write(result.stdout)
    total += 1
pbar.set_description(f'Succesfully proved {success}/{total} Problems')

total, success


Succesfully proved 1554/3123 Problems, currently proving SYN/SYN212-1.p:  35%|███▍      | 3121/8954 [32:51<08:05, 12.02it/s]        

In [8]:
import subprocess
import os
from tqdm.auto import tqdm

outputs = []
for file in tqdm(os.listdir(TPTP_PATH + 'Problems/PUZ/')):
    args = [VAMPIRE_PATH, TPTP_PATH + 'Problems/PUZ/' + file,  '--show_new', 'on', '--include', TPTP_PATH, '-t', '0.1', '--avatar', 'off', '--proof', 'off']
    result = subprocess.run(args, capture_output=True, text=True, timeout=2)
    outputs.append(result.stdout)

 60%|██████    | 146/243 [00:13<00:09, 10.47it/s]


KeyboardInterrupt: 

In [8]:
len(outputs), len([o for o in outputs if 'Refutation' in o])

(243, 161)

In [18]:
from lark import Lark

with open('./foreduce/vampire/vampire.lark', 'r') as f: 
    grammar = Lark(f.read(), parser='lalr')

In [10]:
r1 = grammar.parse(outputs[6])

In [11]:
from lark import Transformer, Token
from foreduce.fol.logic import *
from itertools import chain
from foreduce.fol.derivation import *

class Formulas(Transformer):
    def fof_term(self, children):
        if children[0].type == "FUNCTOR":
            return Function(children[0].value, len(children) - 1)(*children[1:])
        if children[0].type == "VARIABLE":
            return Variable(children[0].value)
    
    def fof_atom(self, children):
        if len(children) == 3 and type(children[1]) == Token:
            return Literal(eq(children[0], children[2]), True)
        else:
            return Literal(Predicate(children[0].value, len(children) - 1)(*children[1:]))

    def fof_negated_atom(self, children):
        children[0].polarity = False
        return children[0]

    def literal(self, children):
        return children[0]

    def disjunction(self, children):
        return Clause(*children)

    def step(self, children):
        return children

    def start(self, children):
        derivations = {}
        for child in children:
            derivations[child[0]] = DerivedClause(
                [derivations[i] for i in child[2][1] if child[2][1][0] in derivations], child[1].literals
            )
        return derivations
    
    def NUMBER(self, token):
        return int(token.value)

    def rule(self, children):
        if len (children) == 1:
            return [children[0], []]
        return children
    
    def LOWER_WORD(self, token):
        return token.value

    def name(self, children):
        return " ".join(chain(children))

    def premises(self, children):
        return children


In [12]:
outputs[0]

'% Running in auto input_syntax mode. Trying TPTP\n[SA] new: 1. c_Finite__Set_Ocard(c_insert(X2,X0,X1),X1) = c_Suc(c_Finite__Set_Ocard(X0,X1)) | c_in(X2,X0,X1) | ~c_in(X0,c_Finite__Set_OFinites,tc_set(X1)) [input]\n[SA] new: 2. c_in(c_inter(X4,X3,X1),c_Finite__Set_OFinites,tc_set(X1)) | ~c_in(X3,c_Finite__Set_OFinites,tc_set(X1)) [input]\n[SA] new: 3. c_in(X5,c_Finite__Set_OFinites,tc_set(tc_prod(tc_nat,tc_nat))) | ~c_in(X5,c_Mutil_Otiling(c_Mutil_Odomino,tc_prod(tc_nat,tc_nat)),tc_set(tc_prod(tc_nat,tc_nat))) [input]\n[SA] new: 4. c_in(X6,X7,X1) | ~c_in(X6,c_inter(X0,X7,X1),X1) [input]\n[SA] new: 5. c_in(X6,c_inter(X0,X7,X1),X1) | ~c_in(X6,X0,X1) | ~c_in(X6,X7,X1) [input]\n[SA] new: 6. c_in(X2,c_insert(X2,X0,X1),X1) [input]\n[SA] new: 7. ~c_in(X6,c_emptyset,X1) [input]\n[SA] new: 8. c_in(v_t,c_Mutil_Otiling(c_Mutil_Odomino,tc_prod(tc_nat,tc_nat)),tc_set(tc_prod(tc_nat,tc_nat))) [input]\n[SA] new: 9. c_Finite__Set_Ocard(c_inter(c_Mutil_Ocoloured(c_0),v_t,tc_prod(tc_nat,tc_nat)),tc_prod

In [14]:
'Refutation found' in  outputs[6]

False

In [15]:
r2 = Formulas().transform(r1)

In [16]:
r2

{1: state(l(n1(), l(n2(), l(n3(), l(n4(), l(end(), l(n5(), l(n6(), l(n7(), l(n8(), l(end(), l(n9(), l(n10(), l(n11(), l(n12(), l(end(), l(n13(), l(n15(), l(n14(), l(x(), l(end(), nil()))))))))))))))))))))),
 2: ¬state(l(n1(), l(n2(), l(n3(), l(n4(), l(end(), l(n5(), l(n6(), l(n7(), l(n8(), l(end(), l(n9(), l(n10(), l(n11(), l(n12(), l(end(), l(n13(), l(n14(), l(n15(), l(x(), l(end(), nil()))))))))))))))))))))),
 3: eq(end(), X0) | eq(l(x(), l(X0, X1)), l(X0, l(x(), X1))),
 4: eq(l(x(), l(X0, l(X2, l(X3, l(X4, l(X5, X1)))))), l(X5, l(X0, l(X2, l(X3, l(X4, l(x(), X1))))))),
 5: eq(l(X2, l(x(), l(x(), X3))), l(x(), l(x(), l(X2, X3)))) | eq(end(), X2) | eq(end(), X2),
 6: eq(l(x(), l(X2, l(X0, X1))), l(X2, l(X0, l(x(), X1)))) | eq(end(), X2) | eq(end(), X0),
 7: eq(l(X2, l(x(), l(x(), X3))), l(x(), l(x(), l(X2, X3)))) | eq(end(), X2),
 8: eq(l(X6, l(X8, l(X9, l(X10, l(X11, l(x(), l(x(), X7))))))), l(x(), l(X8, l(X9, l(X10, l(X11, l(x(), l(X6, X7)))))))) | eq(end(), X6),
 9: eq(l(X22, l(X28

In [39]:
from lark import Lark

with open('./foreduce/vampire/vampire.lark', 'r') as f: 
    grammar = Lark(f.read(), parser='lalr')
Formulas().transform(grammar.parse(outputs[4]))

{1: dom(s(n0())),
 2: dom(s(s(n0()))),
 3: dom(s(s(s(n0())))),
 4: dom(s(s(s(s(n0()))))),
 5: dom(s(s(s(s(s(n0())))))),
 6: dom(s(s(s(s(s(s(n0()))))))),
 7: dom(s(s(s(s(s(s(s(n0())))))))),
 8: dom(s(s(s(s(s(s(s(s(n0()))))))))),
 9: dom(s(s(s(s(s(s(s(s(s(n0())))))))))),
 10: eq(s(n0()), s(s(n0()))),
 11: eq(s(n0()), s(s(s(n0())))),
 12: eq(s(n0()), s(s(s(s(n0()))))),
 13: eq(s(n0()), s(s(s(s(s(n0())))))),
 14: eq(s(n0()), s(s(s(s(s(s(n0()))))))),
 15: eq(s(n0()), s(s(s(s(s(s(s(n0())))))))),
 16: eq(s(n0()), s(s(s(s(s(s(s(s(n0()))))))))),
 17: eq(s(n0()), s(s(s(s(s(s(s(s(s(n0())))))))))),
 18: eq(s(s(n0())), s(s(s(n0())))),
 19: eq(s(s(n0())), s(s(s(s(n0()))))),
 20: eq(s(s(n0())), s(s(s(s(s(n0())))))),
 21: eq(s(s(n0())), s(s(s(s(s(s(n0()))))))),
 22: eq(s(s(n0())), s(s(s(s(s(s(s(n0())))))))),
 23: eq(s(s(n0())), s(s(s(s(s(s(s(s(n0()))))))))),
 24: eq(s(s(n0())), s(s(s(s(s(s(s(s(s(n0())))))))))),
 25: eq(s(s(s(n0()))), s(s(s(s(n0()))))),
 26: eq(s(s(s(n0()))), s(s(s(s(s(n0())))))),
 27:

In [31]:
from torchtune.modules import RotaryPositionalEmbeddings
import torch

rotary = RotaryPositionalEmbeddings(4, base=50)
x = torch.ones(2, 7, 4)
rotary(x.view(2, 7, 1, -1)).view(2, 7, -1)


tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000],
         [-0.3012,  1.3818,  0.8491,  1.1310],
         [-1.3254,  0.4932,  0.6812,  1.2394],
         [-1.1311, -0.8489,  0.4997,  1.3230],
         [ 0.1032, -1.4104,  0.3082,  1.3802],
         [ 1.2426, -0.6753,  0.1106,  1.4099],
         [ 1.2396,  0.6808, -0.0892,  1.4114]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000],
         [-0.3012,  1.3818,  0.8491,  1.1310],
         [-1.3254,  0.4932,  0.6812,  1.2394],
         [-1.1311, -0.8489,  0.4997,  1.3230],
         [ 0.1032, -1.4104,  0.3082,  1.3802],
         [ 1.2426, -0.6753,  0.1106,  1.4099],
         [ 1.2396,  0.6808, -0.0892,  1.4114]]])